In [35]:
import pickle
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn

t_file = open('data/train_data.pkl', 'rb')
train_data = pickle.load(t_file)
v_file = open('data/verify_data.pkl', 'rb')
verify_data = pickle.load(v_file)

users_df = pd.read_csv('data/users.csv')
items_df = pd.read_csv('data/items.csv')

In [70]:
emb_scale = 1024
batch_size = 4096
len_users = len(users_df)
len_items = len(items_df)

In [72]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.feature = data[:,0:2]
        self.label = data[:,2]
        pass
    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    def __len__(self):
        return len(self.feature)
    pass

train_dataset = paddle.io.DataLoader(Dataset(train_data),
                               batch_size=batch_size,
                               shuffle=True)
verify_dataset = paddle.io.DataLoader(Dataset(verify_data),
                               batch_size=batch_size,
                               shuffle=True)                               

In [85]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        return x
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()

In [86]:
dssm.train()
optim = paddle.optimizer.Adam(parameters=dssm.parameters())
for epoch in range(1):
    for batch_id, data in enumerate(train_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')
        y_data = paddle.cast(y_data, dtype='float32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, y_data)
        # acc = paddle.metric.accuracy(y_pred, y_data)
        loss.backward()

        print('epoch:{}, batch_id:{}, loss:{}'.format(epoch, batch_id, loss))
        optim.step()
        optim.clear_grad()

epoch:0, batch_id:0, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.52577114])
epoch:0, batch_id:1, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.27478695])
epoch:0, batch_id:2, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.24525158])
epoch:0, batch_id:3, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.25009668])
epoch:0, batch_id:4, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.23514932])
epoch:0, batch_id:5, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.23011723])
epoch:0, batch_id:6, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.22634792])
epoch:0, batch_id:7, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.22135854])
epoch:0, batch_id:8, loss:Tensor(shape=[1], dtyp

In [89]:
dssm.eval()
for batch_id, data in enumerate(verify_dataset):
    x_data = data[0]
    y_data = data[1]
    x_data = paddle.cast(x_data, dtype='int32')
    y_data = paddle.cast(y_data, dtype='float32')

    y_pred = dssm(x_data)
    loss = nn.functional.mse_loss(y_pred, y_data)
    # acc = paddle.metric.accuracy(y_pred, y_data)

In [92]:
acc = paddle.metric.accuracy(y_pred, y_data)

ValueError: (InvalidArgument) ShapeError: label's dimensions of AccuracyOp must be 2. But received label's dimensions = 1, label's shape = [956]
  [Hint: Expected label_dim.size() == 2, but received label_dim.size():1 != 2:2.] (at C:\home\workspace\Paddle_release\paddle\phi\infermeta\ternary.cc:39)
  [operator < accuracy > error]